In [6]:
import pandas as pd
import numpy as np
from modules import *

In [7]:
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk import word_tokenize
from konlpy.tag import Okt

In [8]:
from nltk.util import everygrams

#원하는 태그 추출 & 불용어 제거
okt = Okt()

#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",
                 '에서','고','이다','젛','뇨','껀데','뭘','빠르다','주문','않다','마시다','구매','받다','말다','오다','정말','넘다',
                 '아직','사다','꼼꼼하다','감사하다','재다','술','보다','아주','먹다','포장','너무','배송','자다','용','생각','더','엄청',
                 '맛','주','산','다','제품','제공','보고','괜찮다','요','많이','추천','달','잇다','마시기','없이','없다','깨지다',
                 '안전하다','먹기','빨대','부담','빙','탄복','나다','항상','자주','박스','최고']) #불용어 추가

stopwords=set(stopwords) # 중복제거    

# 형태소 분석 함수 만들기
def okt_pos_tagging2(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    # n_gram 만들기
    egram = list(everygrams(words, min_len=1, max_len=1))
    egram_token = [' '.join(grams) for grams in egram]

    return egram_token

def okt_pos_tagging(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    return words


In [9]:
# 불러올 csv 이름 리스트
# 전체 리뷰를 불러온후 tf-idf 분석
name_list =['금이산농원_복숭아와인','컨츄리와인_영동포도와인','덕유양조_산머루주','산내들_소백산스위트와인','배상면주가_빙탄복']

In [10]:
all_data = pd.DataFrame()

for name in name_list:
    data = pd.read_csv(f"{name}.csv", encoding='utf-8-sig')
    # 전처리
    data.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거
    data.columns = ['date', 'star','doc']
    data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
    data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
    data.doc = data.doc.str.replace("럽고목","럽고 목")
    data.doc = data.doc.str.replace("부 드럽","부드럽")
    data.dropna(inplace=True)
    data.reset_index(drop=True, inplace=True)
    # globals()["감홍로 40도"] 등으로 데이터 프레임 저장됨
    globals()[f"{name}"] = data
    all_data = pd.concat([all_data,data],axis=0)


In [11]:
all_data

,date,star,doc
0,2022-11-14,5,포장 꼼꼼하게 잘해주셔서 안 깨지고 잘도착했습니다 아직 뜯진 않았는데 병도 너무 예...
1,2022-11-17,4,집들이 때 먹으려고 주문했어요 아직 마셔보지는 않았지만 맛있다해서 기대돼요
2,2022-11-25,5,맛잇어요 적당히 달달해요
3,2022-11-28,5,빠른 배송 감사합니다
4,2022-11-09,5,화이트와인같은느낌이고떫은맛은 전혀없구요당도는 과하지않고 적당지방적은 안심스테이크나 ...
...,...,...,...
7748,2017-08-14,5,만족스럽습니다
7749,2017-07-26,5,빠른배송 맘에 들고요 가족들과 즐기기 좋습니다 시중에서 구하기 힘들었는데 좋네요
7750,2017-08-16,5,맛있네요
7751,2017-08-08,5,기대이상이에요


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=okt_pos_tagging)
tfidf_fit = tfidf.fit(all_data.doc)


In [13]:
# 특징을 추출할 문서의 tf-idf를 구함
a = "배상면주가_빙탄복"
bow_rep_tfidf = tfidf_fit.transform(globals()[a].doc)
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names(),
    'tf-idf': bow_rep_tfidf.sum(axis=0).flat,
    'idf': tfidf.idf_
})


In [14]:
# idf 가 높을수록 잘 안나오는 단어
# tf-idf 가 높을수록 중요한 단어
# 두값을 적절히 조정해서 특징 추출

words = word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>4.3][word_count['tf-idf'] * (len(all_data)/len(globals()[a]))>130]
words

,단어,tf-idf,idf
5359,탄산,188.518848,4.304935
3106,스파클링,156.030861,4.590469
2388,복분자,141.523894,4.657012
1730,만족하다,118.355702,4.492431
4523,적당하다,101.473983,4.565044
3149,시원하다,98.895790,4.991651
2434,부드럽다,83.079966,4.390774
5205,친구,81.400793,4.404934
1282,도수,76.771821,4.592617
637,깔끔하다,67.086394,4.941641


In [16]:
all_data

,date,star,doc
0,2022-11-14,5,포장 꼼꼼하게 잘해주셔서 안 깨지고 잘도착했습니다 아직 뜯진 않았는데 병도 너무 예...
1,2022-11-17,4,집들이 때 먹으려고 주문했어요 아직 마셔보지는 않았지만 맛있다해서 기대돼요
2,2022-11-25,5,맛잇어요 적당히 달달해요
3,2022-11-28,5,빠른 배송 감사합니다
4,2022-11-09,5,화이트와인같은느낌이고떫은맛은 전혀없구요당도는 과하지않고 적당지방적은 안심스테이크나 ...
...,...,...,...
7748,2017-08-14,5,만족스럽습니다
7749,2017-07-26,5,빠른배송 맘에 들고요 가족들과 즐기기 좋습니다 시중에서 구하기 힘들었는데 좋네요
7750,2017-08-16,5,맛있네요
7751,2017-08-08,5,기대이상이에요


In [18]:
#csv  저장
all_data.to_csv("과실주.csv", encoding = 'utf-8-sig')